## Imports

In [ ]:
# Photontorch
import photontorch as pt

# Python
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange

# numpy settings
np.random.seed(6) # seed for random numbers
np.set_printoptions(precision=2, suppress=True) # show less numbers while printing numpy arrays

## Schematic
![Unitary Matrix Paper](images/clements.jpeg)

(a) Reck Design

(b) Clements Design

## Simulation and Design Parameters

Here we will use the matrix network *with* delays.

In [ ]:
N = 4
length = 25e-6 #[m]
transmission = 0.5 #[]
neff = 2.86
env = pt.Environment(
    t_start = 0,
    t_end = 2000e-14,
    dt = 1e-13,
    wl = 1.55e-6,
)
pt.set_environment(env)

source = torch.ones(N, names=["s"])/np.sqrt(N) # Source tensors with less than 4D need to have named dimensions.

env

## A. Reck Design

In [ ]:
nw = pt.ReckNxN(
    N=N, 
    wg_factory=lambda: pt.Waveguide(length=1e-4, phase=2*np.pi*np.random.rand(), trainable=True),
    mzi_factory=lambda: pt.Mzi(length=1e-4, phi=2*np.pi*np.random.rand(), theta=2*np.pi*np.random.rand(), trainable=True),
).terminate()

## Simulation

In [ ]:
detected_time = nw(source)
nw.plot(detected_time[:,0,:,0]); # plot first and only batch

## Total power recovered:

In [ ]:
detected_time[-1,0,:,0].sum()

## Optimizing the coupling

The goal is to optimize the coupling of the network such that we have the same output at the 4 detectors with an as high as possible amplitude (ideally, higher than in the equal coupling case).

In [ ]:
def train_for_same_output(nw, num_epochs=50, learning_rate=0.1):
    target = torch.tensor([0.25,0.25,0.25,0.25], device=nw.device)
    lossfunc = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(nw.parameters(), lr=learning_rate)
    with pt.Environment(wl=1.55e-6, t0=0, t1=10e-12, dt=1e-13, grad=True):
        range_ = trange(num_epochs)
        for epoch in range_:
            det_train = nw(source)[-1,0,:,0]
            loss = lossfunc(det_train, target)
            loss.backward()
            optimizer.step()
            range_.set_postfix(loss=loss.item())
        
train_for_same_output(nw)

## Final Simulation

In [ ]:
%time det_train = nw(source)
nw.plot(det_train[:,0,:,0]); # plot first and only batch

Note that in the Reck network, signals arrive at different times.

## Total power recovered

In [ ]:
det_train[-1,0,:,0].sum()

## B. Clements Design

In [ ]:
nw = pt.ClementsNxN(
    N=N, 
    wg_factory=lambda: pt.Waveguide(length=1e-4, phase=2*np.pi*np.random.rand(), trainable=True),
    mzi_factory=lambda: pt.Mzi(length=1e-4, phi=2*np.pi*np.random.rand(), theta=2*np.pi*np.random.rand(), trainable=True),
).terminate()

## Simulation

In [ ]:
detected_time = nw(source)
nw.plot(detected_time[:,0,:,0]); # plot first and only batch

## Total power recovered:

In [ ]:
detected_time[-1,0,:,0].sum()

## Optimizing the coupling

In [ ]:
def train_for_same_output(nw, num_epochs=50, learning_rate=0.1):
    target = torch.tensor([0.25,0.25,0.25,0.25], device=nw.device)
    lossfunc = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(nw.parameters(), lr=learning_rate)
    with pt.Environment(wl=1.55e-6, t0=0, t1=10e-12, dt=1e-13, grad=True):
        range_ = trange(num_epochs)
        for epoch in range_:
            det_train = nw(source)[-1,0,:,0] # get first and only batch
            loss = lossfunc(det_train, target)
            loss.backward()
            optimizer.step()
            range_.set_postfix(loss=loss.item())
        
train_for_same_output(nw)

## Final Simulation

In [ ]:
%time det_train = nw(source)
nw.plot(det_train[:,0,:,0]); # plot first and only batch

Note that in the Clements network, all signals arrive at the same time at the detector.